In [18]:
import json

dataset = "sg"

data_file = f"{dataset}_merged.json"

data = []

with open(data_file) as f:
    for line in f:
        data.append(json.loads(line))

data = [
    x for x in data
    if "N/A" not in x["task_type"] and "unknown" not in x["task_type"]
]

In [19]:
len(data)

4334

In [20]:
documents = [x['reasoning'] for x in data]

In [21]:
split = [x.strip().split(", ") for x in documents]

In [22]:
combined = [x for y in split for x in y]

In [23]:
from collections import Counter
counts = Counter(combined)

counts

Counter({'problem-solving': 1054,
         'technical knowledge': 566,
         'creativity': 506,
         'analytical skills': 345,
         'logical reasoning': 296,
         'language proficiency': 279,
         'critical thinking': 241,
         'technical analysis': 212,
         'programming skills': 183,
         'creative thinking': 181,
         'research skills': 176,
         'communication skills': 165,
         'strategic planning': 143,
         'information synthesis': 135,
         'storytelling': 125,
         'pattern recognition': 124,
         'information retrieval': 115,
         'cultural awareness': 112,
         'mathematical reasoning': 105,
         'writing skills': 97,
         'programming knowledge': 96,
         'empathy': 95,
         'debugging': 95,
         'research': 84,
         'persuasive writing': 82,
         'critical analysis': 82,
         'coding skills': 80,
         'language understanding': 79,
         'programming logic': 77,
       

In [24]:
import openai
from bertopic.representation import OpenAI

key = ""

# Fine-tune topic representations with GPT
client = openai.OpenAI(api_key=key)

In [28]:

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

representation_model = OpenAI(client, model="gpt-3.5-turbo", chat=True)

# We select a subsample of 5000 abstracts from ArXiv
# docs = [y for x in data for y in x['task_type'].split(", ")]
docs = [x['reasoning'] for x in data]

# We define a number of topics that we know are in the documents
zeroshot_topic_list = [
# "code generation",
# "analysis",
# "literary analysis and interpretation",
# "scientific analysis",
# "data interpretation",
"creativity",
"storytelling",
# "market research",
"problem-solving",
# "creative writing",
# "article writing",
# "email writing",
"language proficiency",
# "technical support",
# "event planning",
# "planning",
# "step-by-step planning",
"analytical skills",
# "strategy",
# "debugging",    
# "brainstorming",
"logical reasoning",
# "health advice",
"advisory skill",
# "design",
# "translation",
# "role-playing",
"communication skills",
# "calculation",
# "summarization",
# "inofrmation retrieval",
"domain knowledge",
"planning",
"empathy",
"critical thinking",
"information synthesis",
# "prompt engineering",
# "evaluation",
# "music composition",
]

# We fit our model using the zero-shot topics
# and we define a minimum similarity. For each document,
# if the similarity does not exceed that value, it will be used
# for clustering instead.
topic_model = BERTopic(
    embedding_model="thenlper/gte-small", 
    min_topic_size=3,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.8,
    representation_model=representation_model
)
topics, _ = topic_model.fit_transform(docs)

In [30]:
topic_model.get_topic_info()[:40]

,Topic,Count,Name,Representation,Representative_Docs
0,0,737,problem-solving,[Problem-Solving with Technical Knowledge],"[ technical knowledge, problem-solving, techni..."
1,1,678,analytical skills,"[problem-solving, research skills, technical k...","[ research skills, technical knowledge, proble..."
2,2,427,creativity,[Creative Marketing and Persuasive Writing Str...,"[creativity, marketing skills, persuasive writ..."
3,3,415,logical reasoning,[Problem-Solving and Logical Reasoning in Tech...,"[technical analysis, problem-solving, logical ..."
4,4,403,language proficiency,[Language Proficiency and Cultural Understandi...,"[ language proficiency, cultural understanding..."
5,5,376,domain knowledge,[Information Retrieval and Technical Knowledge...,"[information retrieval, historical analysis, t..."
6,6,270,critical thinking,[Critical Thinking and Analysis in Research an...,"[technical analysis, problem-solving, critical..."
7,7,241,communication skills,"[communication skills, technical knowledge, pr...","[technical knowledge, problem-solving, communi..."
8,8,206,information synthesis,[Information Synthesis and Data Analysis Techn...,"[information synthesis, research skills, data ..."
9,9,194,storytelling,[Storytelling and Character Development with I...,"[imagination, storytelling skills, character d..."


In [21]:
data_single = [
    x for t,x in zip(topics, data) if t == 0
]

In [22]:
data_single

[{'id': '0',
  'domain': ' religion, philosophy',
  'summary': 'Question about the classification of Buddhism.',
  'task_type': ' philosophical inquiry, religious studies',
  'reasoning': ' critical thinking, knowledge of religious studies, logical reasoning'},
 {'id': '2',
  'domain': ' physics, thermodynamics',
  'summary': 'Question about the rate of ice melting in a specific scenario.',
  'task_type': ' physics problem-solving, thermodynamics analysis',
  'reasoning': ' scientific reasoning, knowledge of thermodynamics, quantitative analysis'},
 {'id': '6',
  'domain': 'engineering, signal processing',
  'summary': 'Provide a method to convert square wave to sine waves.',
  'task_type': 'signal processing, waveform conversion',
  'reasoning': 'mathematical analysis, engineering knowledge'},
 {'id': '9',
  'domain': ' finance, stock market',
  'summary': 'Clarify the pricing convention on the American stock market.',
  'task_type': ' financial inquiry, clarification',
  'reasoning':

In [326]:
Counter(data_single)

Counter({'memory recall': 139,
         'spatial reasoning': 127,
         'mathematical reasoning': 120,
         ' mathematical reasoning': 101,
         'numerical reasoning': 80,
         'conceptual understanding': 74,
         'scientific reasoning': 74,
         ' memory recall': 54,
         ' spatial reasoning': 53,
         'causal reasoning': 49,
         'factual recall': 45,
         'logical reasoning': 42,
         'contextual understanding': 35,
         'quantitative reasoning': 32,
         'fact-based reasoning': 29,
         ' numerical reasoning': 29,
         'historical reasoning': 28,
         'empathy': 25,
         'factual understanding': 25,
         'factual reasoning': 24,
         ' conceptual understanding': 22,
         'biological reasoning': 22,
         'historical understanding': 22,
         'ethical reasoning': 21,
         ' scientific reasoning': 20,
         'fact-checking': 19,
         'geographical reasoning': 19,
         'understanding of 

In [231]:
Counter([y for x in data_single for y in x['reasoning'].split(", ")])

TypeError: string indices must be integers, not 'str'

In [125]:
docs = [x['task_type'] for x in data_single]

zeroshot_topic_list = [
    "information search",
    "recommendation",
]

representation_model = OpenAI(client, model="gpt-3.5-turbo", chat=True)

topic_model_single = BERTopic(
    embedding_model="thenlper/gte-small", 
    min_topic_size=5,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.75,
    representation_model=representation_model
)
topics_single, _ = topic_model_single.fit_transform(docs)

In [126]:
topic_model_single.get_topic_info()[:40]

,Topic,Count,Name,Representation,Representative_Docs
0,0,5167,information search,[Information Retrieval and Inquiry],"[information retrieval, explanation, informat..."
1,1,75,recommendation,[recommendation and information retrieval],"[ information retrieval, recommendation, reco..."


In [82]:
topic_model_single.get_topic_info(4)

,Topic,Count,Name,Representation,Representative_Docs
0,4,13,1_Personal storytelling and reflection for cre...,[Personal storytelling and reflection for crea...,"[ personal reflection, storytelling, personal..."


In [31]:
topic_model.save(f"{dataset}_topic_model_reasoning", serialization="pytorch")
# with open(f"{dataset}_topics_2.json", "w") as f:
#     json.dump(topics, f)